# Retiree Income Projection

In [38]:
import pandas as pd

# Constants
SAVINGS_INTEREST = 0.06
IRA_INTEREST = 0.08
INITIAL_SAVINGS = 580  # Initial savings in thousands of dollars
INITIAL_IRA_BALANCE = 400  # Initial IRA balance in thousands of dollars
BASE_SPENDING = 81.5  # Base annual spending in thousands of dollars
MA_TAX_RATE = 0.05  # Massachusetts flat tax rate
SIMULATION_YEARS = 29
START_AGE = 62  # The age at which the simulation starts
CURRENT_YEAR = 2023  # The current year where the simulation starts

# Load CSV files
fed_tax_df = pd.read_csv('./data/federal.csv')
cl_tax_df = pd.read_csv('./data/cltax.csv')
inflation_rate_df = pd.read_csv('./data/inflation.csv')
life_expectancy_df = pd.read_csv('./data/rmd.csv')
mass_tax_df = pd.read_csv('./data/masstax.csv')

# Tax calculation function that uses a flat rate or brackets
def calculate_tax(income, brackets_df, flat_rate=None):
    if flat_rate is not None:
        # Flat tax calculation
        return income * flat_rate
    else:
        # Tax computation that is progressive
        brackets_df = brackets_df.sort_values(by='Income')
        tax_paid = 0
        for _, bracket in brackets_df.iterrows():
            if income > bracket['Income']:
                tax_paid = bracket['Base Tax']
                tax_paid += (income - bracket['Income']) * bracket[' percentage'] / 100
                break
        return tax_paid

# Retirement simulation function
def simulate_retirement(savings, ira_balance, base_spending, savings_interest_rate, ira_interest_rate, federal_brackets, state_brackets, inflation_rates, life_expectancy, current_age, years, current_year):
    results = []
    ira_depleted_year = None  # To record the year that an IRA is exhausted
    last_year_income = 0  # Income used in tax computations from the previous year

    for i in range(years):
        year = current_year + i

        # Determine the interest you've earned.
        savings_interest = savings * savings_interest_rate
        ira_interest = ira_balance * ira_interest_rate
        total_income = savings_interest + ira_interest  # Total income without RMD

        # If necessary, determine the Required Minimum Distributions (RMD).
        rmd = 0
        if current_age + i >= 72:  # Assuming RMD starts at 72
            life_exp_entry = life_expectancy[life_expectancy['Age'] == current_age + i]
            if not life_exp_entry.empty:
                distribution_period = life_exp_entry.iloc[0]['Distribution period']
                rmd = ira_balance / distribution_period
                total_income += rmd  # Adjust total income to account for RMD

        # Use last year's income to compute federal and state taxes.
        federal_tax = 0
        state_tax = 0
        if year > CURRENT_YEAR:
            federal_tax = calculate_tax(last_year_income, federal_brackets) if federal_brackets is not None else 0
            state_tax = calculate_tax(last_year_income, None, flat_rate=MA_TAX_RATE)

        # Update balances
        savings += savings_interest
        ira_balance += ira_interest - rmd

        # Make inflation adjustments to base spending
        inflation_rate = inflation_rates.iloc[i]['Percentage Change'] / 100
        inflation_amount = base_spending * inflation_rate

        # Determine the annual spending total.
        total_spending = base_spending + inflation_amount + federal_tax + state_tax

        # Determine if IRA spending is required
        ira_spending = 0
        if savings >= total_spending:
            savings -= total_spending
        else:
            ira_spending = total_spending - savings
            ira_balance -= ira_spending
            savings = 0
            # Check if the IRA is depleted
            if ira_balance < 0 and ira_depleted_year is None:
                ira_depleted_year = year

        # Enclose the current year's results.
        results.append({
            'Year': year,
            'Savings': max(savings, 0),  # Ensure savings don't go below zero
            'IRA Balance': max(ira_balance, 0),  # Ensure IRA balance doesn't go below zero
            'Total Spending': total_spending,
            'Federal Tax': federal_tax,
            'State Tax': state_tax,
            'RMD': rmd,
            'IRA Spending': ira_spending
        })

        # Adjust income from previous year to compute taxes for the following year.
        last_year_income = total_income

    return pd.DataFrame(results), ira_depleted_year

In [39]:
# Execute the California simulation.
ca_results, ca_ira_depleted_year = simulate_retirement(
    INITIAL_SAVINGS,
    INITIAL_IRA_BALANCE,
    BASE_SPENDING,
    SAVINGS_INTEREST,
    IRA_INTEREST,
    fed_tax_df,
    cl_tax_df,
    inflation_rate_df,
    life_expectancy_df,
    START_AGE,
    SIMULATION_YEARS,
    CURRENT_YEAR
)

# Present the outcomes
print("California Simulation Results:")
print(ca_results)
print(f"Year when IRA gets depleted in California: {ca_ira_depleted_year if ca_ira_depleted_year else 'IRA not depleted in simulation period'}")

California Simulation Results:
    Year     Savings  IRA Balance  Total Spending  Federal Tax  State Tax  \
0   2023  533.234800   432.000000       81.565200     0.000000   0.000000   
1   2024  480.281338   466.560000       84.947550     0.066800   3.340000   
2   2025  424.179510   503.884800       84.918708     0.066554   3.327704   
3   2026  364.740755   544.195584       84.889526     0.066142   3.307084   
4   2027  301.755061   587.731231       84.870139     0.065762   3.288078   
5   2028  235.007640   634.749729       84.852725     0.065420   3.271005   
6   2029  164.270484   685.529708       84.837614     0.065124   3.256190   
7   2030   89.301511   740.372084       84.825202     0.064880   3.244022   
8   2031    9.843673   799.601851       84.815929     0.064699   3.234930   
9   2032    0.000000   789.194011       84.810281     0.064588   3.229393   
10  2033    0.000000   738.718034       84.808797     0.064559   3.227938   
11  2034    0.000000   683.734175       86.20

In [40]:
import pandas as pd

# Constants
SAVINGS_INTEREST = 0.06
IRA_INTEREST = 0.08
INITIAL_SAVINGS = 580  # Initial savings in thousands of dollars
INITIAL_IRA_BALANCE = 400  # Initial IRA balance in thousands of dollars
BASE_SPENDING = 81.5  # Base annual spending in thousands of dollars
MA_TAX_RATE = 0.05  # Massachusetts flat tax rate
SIMULATION_YEARS = 30  # Extend simulation to 30 years to ensure IRA depletion by the end of 2039
START_AGE = 62  # The age at which the simulation starts
CURRENT_YEAR = 2023  # The current year where the simulation starts

# Load CSV files
fed_tax_df = pd.read_csv('./data/federal.csv')
cl_tax_df = pd.read_csv('./data/cltax.csv')
inflation_rate_df = pd.read_csv('./data/inflation.csv')
life_expectancy_df = pd.read_csv('./data/rmd.csv')
mass_tax_df = pd.read_csv('./data/masstax.csv')

# Tax calculation function that uses a flat rate or brackets
def calculate_tax(income, brackets_df, flat_rate=None):
    if flat_rate is not None:
        # Flat tax calculation
        return income * flat_rate
    else:
        # Tax computation that is progressive
        brackets_df = brackets_df.sort_values(by='Income', ascending=False)
        tax_paid = 0
        for _, bracket in brackets_df.iterrows():
            if income > bracket['Income']:
                tax_paid = bracket['Base Tax']
                tax_paid += (income - bracket['Income']) * bracket[' percentage'] / 100
                break
        return tax_paid

# Retirement simulation function for Massachusetts
def simulate_retirement_ma(savings, ira_balance, base_spending, savings_interest_rate, ira_interest_rate, federal_brackets, inflation_rates, life_expectancy, current_age, years, current_year):
    results = []
    ira_depleted_year = None  # To record the year that an IRA is exhausted
    cumulative_federal_tax = 0  # Cumulative federal tax
    cumulative_state_tax = 0  # Cumulative state tax

    for i in range(years):
        year = current_year + i
        age = START_AGE + i

        # Determine the interest you've earned.
        savings_interest = savings * savings_interest_rate
        ira_interest = ira_balance * ira_interest_rate
        total_income = savings_interest + ira_interest  # Total income without RMD

        # If necessary, determine the Required Minimum Distributions (RMD).
        rmd = 0
        if current_age + i >= 72:  # Assuming RMD starts at 72
            life_exp_entry = life_expectancy[life_expectancy['Age'] == current_age + i]
            if not life_exp_entry.empty:
                distribution_period = life_exp_entry.iloc[0]['Distribution period']
                rmd = ira_balance / distribution_period
                total_income += rmd  # Update total income to include RMD

        # Use last year's income to compute federal and state taxes.
        federal_tax = calculate_tax(total_income, federal_brackets) if federal_brackets is not None else 0
        state_tax = total_income * MA_TAX_RATE  # Massachusetts flat tax rate

        # Update cumulative taxes
        cumulative_federal_tax += federal_tax
        cumulative_state_tax += state_tax

        # Update balances
        savings += savings_interest
        ira_balance += ira_interest - rmd

        # Make inflation adjustments to base spending
        inflation_rate = inflation_rates.iloc[i]['Percentage Change'] / 100
        inflation_amount = base_spending * inflation_rate

        # Calculate total spending for the year
        total_spending = base_spending + inflation_amount + cumulative_federal_tax + cumulative_state_tax

        # Determine if IRA spending is required
        ira_spending = 0
        if savings >= total_spending:
            savings -= total_spending
        else:
            ira_spending = total_spending - savings
            ira_balance -= ira_spending
            savings = 0
            # Check if the IRA is depleted
            if ira_balance <= 0 and ira_depleted_year is None:
                ira_depleted_year = year

        # Enclose the current year's results.
        results.append({
            'Year': year,
            'Age': age,
            'Savings': max(savings, 0),  # Ensure savings don't go below zero
            'IRA Balance': max(ira_balance, 0),  # Ensure IRA balance doesn't go below zero
            'Total Spending': total_spending,
            'Federal Tax': federal_tax,
            'State Tax': state_tax,
            'RMD': rmd,
            'IRA Spending': ira_spending
        })

    return pd.DataFrame(results), ira_depleted_year

# Execute the massachusetts simulation.
ma_simulation_results, ma_ira_depleted_year = simulate_retirement_ma(
    INITIAL_SAVINGS,
    INITIAL_IRA_BALANCE,
    BASE_SPENDING,
    SAVINGS_INTEREST,
    IRA_INTEREST,
    fed_tax_df,
    inflation_rate_df,
    life_expectancy_df,
    START_AGE,
    SIMULATION_YEARS,
    CURRENT_YEAR
)

# Display the results for Massachusetts
print("Massachusetts Simulation Results:")
print(ma_simulation_results)
print(f"Year when IRA gets depleted in Massachusetts: {ma_ira_depleted_year if ma_ira_depleted_year else 'IRA not depleted in simulation period'}")

Massachusetts Simulation Results:
    Year  Age     Savings  IRA Balance  Total Spending  Federal Tax  \
0   2023   62  529.828000   432.000000       84.972000     0.066800   
1   2024   63  473.286296   466.560000       88.331384     0.066350   
2   2025   64  410.016570   503.884800       91.666905     0.065722   
3   2026   65  339.648308   544.195584       94.969255     0.064912   
4   2027   66  261.798310   587.731231       98.228897     0.063915   
5   2028   67  176.078265   634.749729      101.427943     0.062726   
6   2029   68   82.086440   685.529708      104.556522     0.061345   
7   2030   69    0.000000   719.779043      107.604667     0.059768   
8   2031   70    0.000000   666.820000      110.541366     0.057582   
9   2032   71    0.000000   606.903609      113.261991     0.053346   
10  2033   72    0.000000   516.438334      116.867796     0.070702   
11  2034   73    0.000000   418.296397      119.968765     0.060803   
12  2035   74    0.000000   312.844322     

# Assumptions

**Life Expectancy:** The retiree's current age is not mentioned, let's assume the retiree is around 65 years old.

**Investment Returns:** For simplicity, we'll assume a constant annual return on investment for both the IRA and Savings Account. Let's assume a conservative 5% annual return for the IRA and 3% for the Savings Account.

**Minimum Required Distributions (RMDs):** Since the tax accountant mentioned recent changes and provided an outdated table, we'll assume the retiree will start RMDs at 72 as per current IRS guidelines. We'll also assume the retiree will initially withdraw only the minimum required amount.

**Inflation:** We'll use the provided inflation table for projections, assuming the given rates are accurate.

**Expense Stability:** We'll assume that the retiree's annual spending budget of $81,500 remains constant, with adjustments for inflation.

**Tax Brackets:** We'll use the tax bracket information provided for both Massachusetts and California. We'll assume no significant changes in tax laws during the retirement period.

# References

IRS guidelines for Required Minimum Distributions (RMDs).

Massachusetts and California tax bracket information.

Provided inflation table for projections.

# Demo Video 

Youtube Link 
https://youtu.be/tO0p2tr1II8